In [ ]:
import yfinance as yf
import pandas as pd

# 1. LISTA DE TICKERS
tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'TSLA', 'NVDA', 'META', 'UNH', 'JPM', 'V', 'XOM', 'LLY', 'PG', 'MA', 'AVGO']

print("--- Iniciando proceso ETL desde Yahoo Finance ---")

# 2. DEFINICIÓN DE SECTORES
sectores_data = [
    (1, 'Tecnologia de la Informacion', 'Empresas de software, hardware y servicios digitales', 'Software y Servicios de TI', 'Global', 'S&P 500 Information Technology', 'Medio'),
    (2, 'Salud', 'Companias farmaceuticas, biotecnologia y servicios medicos', 'Farmaceuticas y Biotecnologia', 'Global', 'S&P 500 Health Care', 'Medio'),
    (3, 'Financieros', 'Bancos, aseguradoras y servicios financieros', 'Banca y Seguros', 'Global', 'S&P 500 Financials', 'Medio-Alto'),
    (4, 'Energia', 'Empresas de petroleo, gas y energias alternativas', 'Petroleo y Gas', 'Global', 'S&P 500 Energy', 'Alto'),
    (5, 'Materiales', 'Productores de quimicos, metales y materiales basicos', 'Quimicos y Metales', 'Global', 'S&P 500 Materials', 'Medio-Alto'),
    (6, 'Industriales', 'Fabricacion, transporte y servicios industriales', 'Aeroespacial y Transporte', 'Global', 'S&P 500 Industrials', 'Medio'),
    (7, 'Consumo Discrecional', 'Bienes y servicios no esenciales como autos, retail y lujo', 'Retail y Automotriz', 'Global', 'S&P 500 Consumer Discretionary', 'Medio'),
    (8, 'Consumo Basico', 'Alimentos, bebidas y productos esenciales', 'Alimentos y Bebidas', 'Global', 'S&P 500 Consumer Staples', 'Bajo'),
    (9, 'Servicios de Comunicacion', 'Telecomunicaciones, medios y entretenimiento digital', 'Telecomunicaciones y Medios', 'Global', 'S&P 500 Communication Services', 'Medio'),
    (10, 'Servicios Publicos', 'Electricidad, agua y gas', 'Utilities', 'Global', 'S&P 500 Utilities', 'Bajo'),
    (11, 'Bienes Raices', 'REITs y desarrolladoras inmobiliarias', 'Real Estate Investment Trusts', 'Global', 'S&P 500 Real Estate', 'Medio')
]
df_sectores = pd.DataFrame(sectores_data, columns=['sector_id', 'Sector', 'Descripcion', 'Industria', 'Region', 'benchmark_index', 'riesgo_relativo'])

# 3. EXTRACCIÓN DE DATOS DE ACTIVOS
info_list = []
mapeo_ingles = {
    'Technology': 1, 'Healthcare': 2, 'Financial Services': 3, 'Energy': 4,
    'Basic Materials': 5, 'Industrials': 6, 'Consumer Cyclical': 7,
    'Consumer Defensive': 8, 'Communication Services': 9, 'Utilities': 10, 'Real Estate': 11
}

for t in tickers:
    try:
        dat = yf.Ticker(t).info
        dat['symbol'] = t
        info_list.append(dat)
    except:
        continue

df_raw = pd.DataFrame(info_list)

# 4. GENERACIÓN DE ARCHIVOS

# A. Sectores
df_sectores.to_csv('1_Sectores_Final.csv', index=False)

# B. Activos (Vinculando sector_id)
df_activos = df_raw[['symbol', 'longName', 'typeDisp', 'sector', 'country', 'financialCurrency']].copy()
df_activos['sector_id'] = df_activos['sector'].map(mapeo_ingles).fillna(0).astype(int)
df_activos[['symbol', 'longName', 'typeDisp', 'sector_id', 'country', 'financialCurrency']].to_csv('2_Activos_Final.csv', index=False)

# C. Desempeño
df_desemp = df_raw[['symbol', 'currentPrice', 'previousClose', 'dayLow', 'dayHigh', 'marketCap', 'beta']].copy()
df_desemp['rendimiento'] = (df_desemp['currentPrice'] - df_desemp['previousClose']) / df_desemp['previousClose']
df_desemp.to_csv('3_Desempeno_Final.csv', index=False)

# D. Precios (Histórico 4 años)
print("--- Descargando históricos (esto puede tardar un poco) ---")
df_h = yf.download(tickers, period="4y", interval="1d").stack().reset_index()
df_h.columns = ['Fecha', 'Simbolo', 'Cierre', 'Apertura', 'Maximo', 'Minimo', 'Volumen']
df_h['ID'] = df_h['Simbolo'] + "_" + df_h['Fecha'].dt.strftime('%Y-%m-%d')
df_h[['ID', 'Simbolo', 'Fecha', 'Apertura', 'Cierre', 'Maximo', 'Minimo', 'Volumen']].to_csv('4_Precios_Final.csv', index=False)

print("--- ¡Listo! Los 4 archivos ---")

--- Iniciando proceso ETL desde Yahoo Finance ---
--- Descargando históricos (esto puede tardar un poco) ---


/tmp/ipython-input-3090496197.py:60: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df_h = yf.download(tickers, period="4y", interval="1d").stack().reset_index()
[*********************100%***********************]  15 of 15 completed
/tmp/ipython-input-3090496197.py:60: FutureWarning: The previous implementation of stack is deprecated and will be removed in a future version of pandas. See the What's New notes for pandas 2.1.0 for details. Specify future_stack=True to adopt the new implementation and silence this warning.
  df_h = yf.download(tickers, period="4y", interval="1d").stack().reset_index()


--- ¡Listo! Los 4 archivos están en la carpeta de la izquierda ---


In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

print("--- Iniciando Generación de Datos Sistematicos ---")

# 1. GENERACIÓN DE TABLA 5: PORTAFOLIOS (Estrategias Optimizadas)
portafolios_data = [
    # Moderado (Equilibrado)
    ('PORT_MODERADO', 'AAPL', 0.20), ('PORT_MODERADO', 'MSFT', 0.20),
    ('PORT_MODERADO', 'JPM', 0.20), ('PORT_MODERADO', 'UNH', 0.20), ('PORT_MODERADO', 'XOM', 0.20),
    # Agresivo (Crecimiento/Tecnología)
    ('PORT_AGRESIVO', 'NVDA', 0.35), ('PORT_AGRESIVO', 'TSLA', 0.25),
    ('PORT_AGRESIVO', 'META', 0.20), ('PORT_AGRESIVO', 'AMZN', 0.20),
    # Conservador (Baja Volatilidad)
    ('PORT_CONSERVADOR', 'PG', 0.40), ('PORT_CONSERVADOR', 'V', 0.30), ('PORT_CONSERVADOR', 'LLY', 0.30)
]
df_port = pd.DataFrame(portafolios_data, columns=['portafolio_id', 'symbol', 'peso'])
df_port.to_csv('5_Portafolios.csv', index=False)

# 2. GENERACIÓN DE TABLA 6: INVERSIONISTAS
inversionistas_data = [
    (1, 'Ingeniero', 100000.00), (2, 'Analista', 50000.00), (3, 'Fondo_Inv', 500000.00),
    (4, 'Retail_01', 15000.00), (5, 'Wealth_Mgmt', 1000000.00), (6, 'Pension_Fd', 2000000.00),
    (7, 'Family_Off', 750000.00), (8, 'Trading_Bot', 25000.00), (9, 'Uni_Endow', 300000.00),
    (10, 'Venture_Cap', 450000.00)
]
df_inv = pd.DataFrame(inversionistas_data, columns=['inversionista_id', 'nombre', 'capital_total'])
df_inv.to_csv('6_Inversionistas.csv', index=False)

# 3. GENERACIÓN DE TABLA 7: TRANSACCIONAL
port_ids = ['PORT_MODERADO', 'PORT_AGRESIVO', 'PORT_CONSERVADOR']
transacciones_list = []
t_id = 1
fecha_inicio = datetime.now() - timedelta(days=45)

for inv_id, nombre, capital in inversionistas_data:
    # Cada inversionista realiza entre 5 y 12 operaciones
    for _ in range(random.randint(5, 12)):
        monto = round(random.uniform(1000, capital * 0.12), 2)
        tipo = random.choices(['COMPRA', 'VENTA'], weights=[0.85, 0.15])[0]
        port = random.choice(port_ids)

        # Generar fecha aleatoria con dispersión horaria
        fecha = (fecha_inicio + timedelta(days=random.randint(0, 45), minutes=random.randint(0, 1440))).strftime('%Y-%m-%d %H:%M:%S')

        transacciones_list.append((t_id, inv_id, port, fecha, tipo, monto))
        t_id += 1

df_trans = pd.DataFrame(transacciones_list, columns=['transaccion_id', 'inversionista_id', 'portafolio_id', 'fecha', 'tipo_operacion', 'monto'])
df_trans = df_trans.sort_values(by='fecha')
df_trans.to_csv('7_Transaccional.csv', index=False)

print(f"Archivos creados: 5_Portafolios.csv, 6_Inversionistas.csv, 7_Transaccional.csv")
print(f"Total de transacciones simuladas: {len(df_trans)}")

--- Iniciando Generación Consolidada de Capa Operativa (BI) ---
Archivos creados: 5_Portafolios.csv, 6_Inversionistas.csv, 7_Transaccional.csv
Total de transacciones simuladas: 84
